In [1]:
import sys
import argparse
import os
import torch
from collections import OrderedDict
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import seaborn as sns
import pylab as py
import time
from pyDOE import lhs
import warnings
sys.path.insert(0, '../../../Scripts/')
from models_imperfect import PGNN
from pinn import *
# from ../Scripts/helper import *

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda:3')
else:
    device = torch.device('cpu')

## Hyper-parameters

In [3]:
num_epochs = 10000
lambda_val = 1
tr_frac = 0.4

#architecture for the models
net_hid_dim = 50
net_num_layer = 4

### Load Data

In [4]:
data = np.loadtxt('../../../datasets/tossing_trajectories.txt')
x = data[:, 0:6]
labels = data[:, 6:]

# training and test split
n_obs = int(tr_frac * x.shape[0])
train_x , train_y = x[:n_obs,:] , labels[:n_obs, :] 
test_x , test_y = x[n_obs:,:] , labels[n_obs:, :]

data_dim = train_x.shape[1]
out_dim = labels.shape[1]

#### Models

In [5]:
net = PGNN(in_dim = data_dim, out_dim = out_dim, hid_dim=net_hid_dim, num_layers=net_num_layer).to(device)

In [6]:
PINN = Tossing_PINN(train_x, train_y, test_x, test_y, net, device, num_epochs, lambda_val)

### Training

In [7]:
PINN.train()

[Epoch 0/10000] [MSE loss: 1.005757] [Phy loss: 111.047653] [Total loss: 112.053411]
[Epoch 100/10000] [MSE loss: 0.066064] [Phy loss: 8.648100] [Total loss: 8.714163]
[Epoch 200/10000] [MSE loss: 0.069571] [Phy loss: 5.922823] [Total loss: 5.992394]
[Epoch 300/10000] [MSE loss: 0.055163] [Phy loss: 5.524343] [Total loss: 5.579506]
[Epoch 400/10000] [MSE loss: 0.052859] [Phy loss: 5.066592] [Total loss: 5.119451]
[Epoch 500/10000] [MSE loss: 0.050131] [Phy loss: 4.833365] [Total loss: 4.883495]
[Epoch 600/10000] [MSE loss: 0.063434] [Phy loss: 4.741381] [Total loss: 4.804815]
[Epoch 700/10000] [MSE loss: 0.042832] [Phy loss: 4.510007] [Total loss: 4.552840]
[Epoch 800/10000] [MSE loss: 0.075011] [Phy loss: 4.587468] [Total loss: 4.662480]
[Epoch 900/10000] [MSE loss: 0.047799] [Phy loss: 4.562764] [Total loss: 4.610564]
[Epoch 1000/10000] [MSE loss: 0.063854] [Phy loss: 4.468031] [Total loss: 4.531885]
[Epoch 1100/10000] [MSE loss: 0.055971] [Phy loss: 4.281302] [Total loss: 4.337273]


In [8]:
nsamples = 500
train_mean_y, train_std_y = PINN.uncertainity_estimate(PINN.train_x, nsamples, [PINN.Ymean, PINN.Ystd])

train_predictions = torch.Tensor(train_mean_y).float().to(device)

std_y_t = torch.Tensor(PINN.Ystd).float().to(device)
mean_y_t = torch.Tensor(PINN.Ymean).float().to(device)
std_x_t = torch.Tensor(PINN.Xstd).float().to(device)
mean_x_t = torch.Tensor(PINN.Xmean).float().to(device)

train_y = (PINN.train_y * std_y_t) + mean_y_t
train_x = (PINN.train_x * std_x_t) + mean_x_t

mse_train = ((train_predictions - train_y)**2).mean()
phy_train = torch.mean(PINN.physics_loss(train_x, train_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Training Set : ", mse_train.detach().cpu().numpy())
print("Physical Consistency: ", phy_train)

MSE on Training Set :  0.638994
Physical Consistency:  0.5296485


In [9]:
test_mean_y, test_std_y = PINN.uncertainity_estimate(PINN.test_x, nsamples, [PINN.Ymean, PINN.Ystd])

test_predictions = torch.Tensor(test_mean_y).float().to(device)

test_y = (PINN.test_y * std_y_t) + mean_y_t
test_x = (PINN.test_x * std_x_t) + mean_x_t

mse_test = ((test_predictions - test_y)**2).mean()
phy_test = torch.mean(PINN.physics_loss(test_x, test_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Test Set : ", mse_test.detach().cpu().numpy())
print("Physical Consistency: ", phy_test)

MSE on Test Set :  0.7067647
Physical Consistency:  0.5398416
